In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

# 開口部の日射熱取得率を計算するモジュール

## 1. Functions  

### 1.1. 時刻別

グレージング複合体の日射熱取得率、フレームの日射熱取得率、グレージング面積比又はグレージング面積及びフレーム面積から開口部の日射熱取得率を計算する。

### 入力値  

$ \eta_{glz} $ ：グレージング複合体の日射熱取得率  
$ \eta_{fr} $ ：フレームの日射熱取得率   
$ F_{Ag} $ ：グレージング面積比   
または、  
$ A_{glz} $ ：グレージング面積 (m<sup>2</sup>)   
$ A_{fr} $ ：フレーム面積 (m<sup>2</sup>)   

### 出力値  

$ \eta_{w} $ ：開口部の日射熱取得率  

### 計算方法  

#### データ構造と値の範囲  

$ \eta_{glz} $・$ \eta_{fr} $・im・fm・$ A_{glz} $・$ A_{fr} $の組み合わせをクラス『ghw＿input』と定義する。  

In [2]:
class ghw_input():
    def __init__( self, gg, gf, im, fm, Ag, Af ):
        self.gg = gg
        self.gf = gf
        self.im = im
        self.fm = fm
        self.Ag = Ag
        self.Af = Af
        
#gg：グレージング複合体の日射熱取得率
#gf：フレームの日射熱取得率
#im：グレージング面積比のデータ取得方法（0：表参照入力、1：直接入力）
#fm：フレーム材質のID（0：木製建具又は樹脂製建具、
#                      1：木と金属の複合材料製建具、樹脂と金属の複合材料製建具、金属製熱遮断構造建具又は金属製建具）
#                    （※imが1の場合は値を無視）
#Ag：グレージング面積 (m2)（※imが0の場合は値を無視）
#Af：フレーム面積 (m2)（※imが0の場合は値を無視）

#### 開口部の日射熱取得率の計算方法  

開口部の日射熱取得率$\eta_{w}$の計算方法を以下に示す。  

$$
\begin{eqnarray}
 &\displaystyle \eta_{w} = F_{Ag} \cdot \eta_{glz} + (1 - F_{Ag}) \cdot \eta_{fr}& \qquad\qquad\text{(1)} \nonumber\\
\end{eqnarray}
$$

ここで、  
$ \eta_{glz} $ ：グレージング複合体の日射熱取得率  
$ \eta_{fr} $ ：フレームの日射熱取得率  
$ F_{Ag} $ ：グレージング面積比  

$$
\begin{eqnarray}
 &\displaystyle F_{Ag} = \frac {A_{glz}}{A_{glz} + A_{fr}}& \qquad\qquad\text{(2)} \nonumber\\
\end{eqnarray}
$$

ここで、  
$ A_{glz} $ ：グレージング面積 (m<sup>2</sup>)  
$ A_{fr} $ ：フレーム面積 (m<sup>2</sup>)  

である。 

$ F_{Ag} $の代表的な物性値を以下に示す（参考：平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）Ver.2.1 算定方法 3.4　日射熱取得率）。  

| ID   | 材質                                                                                     | グレージング面積比$F_{Ag}$ |
| ---: | :---                                                                                     | ---:                       |
| 0    | 木製建具又は樹脂製建具                                                                   | 0.72                       |
| 1    | 木と金属の複合材料製建具、樹脂と金属の複合材料製建具、金属製熱遮断構造建具又は金属製建具 | 0.80                       |

In [3]:
def hourly_gvalue_window(L):
    if L.im == 0:     #表参照入力
        F_Ag = glazing_area_ratio(L)
    elif L.im == 1:   #直接入力
        F_Ag = L.Ag / (L.Ag + L.Af)
    
    return F_Ag * L.gg + (1. - F_Ag) * L.gf

def glazing_area_ratio(L):
    
    r_F_Ag = [0.8, 0.72]
    
    return r_F_Ag[L.fm]

#### Example

In [4]:
if __name__ == '__main__':

    ghw_in = ghw_input(0.65, 0.05, 0, 0, 0, 0)

    ghw = hourly_gvalue_window(ghw_in)

    print(ghw)

0.53
